# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import requests

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/'

fileNames = ["dom.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)



In [ ]:
dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
dom_gauss = cv2.GaussianBlur(dom, (5, 5), 0)
dom_laplacian = cv2.Laplacian(dom_gauss, cv2.CV_32F)
f, axs = plt.subplots(1, 3, figsize=(20,10))
axs[0].imshow(dom, 'grey')
axs[0].axis('off')
axs[1].imshow(dom_gauss, 'grey')
axs[1].axis('off')
axs[2].imshow(dom_laplacian, 'grey')
axs[2].axis('off')
plt.show()

In [ ]:
def crossing(LoG, thr):
  img = np.zeros(LoG.shape)
  for x in range(1, LoG.shape[0] - 1):
    for y in range(1, LoG.shape[1] - 1):
      window = LoG[x-1:x+2, y-1:y+2]
      min = np.min(window)
      max = np.max(window)
      if min * max < 0:
        if LoG[x, y] > 0:
          img[x, y] = LoG[x, y] + abs(min)
        if LoG[x, y] < 0:
          img[x, y] = abs(LoG[x, y]) + max
  img_norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
  img_binary = np.where(img_norm > thr, 255, 0)
  img_binary = img_binary * 255
  img_binary = img_binary.astype(np.uint8)
  img_binary = cv2.medianBlur(img_binary, 5)
  return img_binary

In [ ]:
dom_crossing = crossing(dom_laplacian, 50)
f, axs = plt.subplots(1, 2, figsize=(20,10))
axs[0].imshow(dom, 'grey')
axs[0].axis('off')
axs[1].imshow(dom_crossing, 'grey')
axs[1].axis('off')
plt.show()

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
import scipy
def canny(image, high, low):
  img_gauss = cv2.GaussianBlur(image, (3, 3), 0)
  S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
  S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
  img_x = scipy.signal.convolve2d(img_gauss, S1, mode="same")
  img_y = scipy.signal.convolve2d(img_gauss, S2, mode="same") #filtracja sobela
  img_mag = np.sqrt(img_x ** 2 + img_y ** 2) #amplituda
  img_angle = np.rad2deg(np.arctan2(img_y, img_x)) #kąt
  img_angle[img_angle < 0] += 180 #na stopnie

  img_nonmax = np.zeros((image.shape[0], image.shape[1]))
  for x in range(image.shape[0]):
    for y in range(image.shape[1]):
      if (0 <= img_angle[x, y] < 22.5) or (157.5 <= img_angle[x, y] <= 180): #poziomy
        img_nonmax[x, y] = 0
      elif (22.5 <= img_angle[x, y] < 67.5): #skosny prawy
        img_nonmax[x, y] = 1
      elif (67.5 <= img_angle[x, y] < 112.5): #pionowy
        img_nonmax[x, y] = 2
      elif (112.5 <= img_angle[x, y] < 157.5): #skosny lewy
        img_nonmax[x, y] = 3

  img_nonmax = nonmax(img_nonmax, img_mag)
  img_nh = img_nonmax >= high #pewne krawedzie
  img_nl = (img_nonmax >= low) & (img_nonmax < high) #potencjalne krawedzie

  stos_nh = []
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if img_nh[i, j] == 1: #jesli jest pewna krawedzia
        stos_nh.append([i, j])
  visited = np.zeros((image.shape[0], image.shape[1]))
  img_result = np.zeros((image.shape[0], image.shape[1]))
  while stos_nh:
    i = stos_nh.pop()
    if visited[i[0], i[1]] == 0: #jesli nieodwiedzony
      visited[i[0], i[1]] = 1
      img_result[i[0], i[1]] = 1 #dodaj do krawedzi koncowych
      for x in range(- 1, 2):
        for y in range(- 1, 2):
          if img_nl[i[0] + x, i[1] + y] == 1: #jesli otoczenie w krawedziach potencjalnych
            stos_nh.append([i[0] + x, i[1] + y])
  return img_result

def nonmax(matrix, img_mag): #zeby krawedz na 1 piksel
  img_nonmax = np.zeros((matrix.shape[0], matrix.shape[1]))
  for x in range(1, matrix.shape[0] - 1):
    for y in range(1, matrix.shape[1] - 1):
      if matrix[x, y] == 0: #poziomy
        if (img_mag[x, y] > img_mag[x, y - 1]) and (img_mag[x, y] > img_mag[x, y + 1]):
          img_nonmax[x, y] = img_mag[x, y]
        else:
          img_nonmax[x, y] = 0
      elif matrix[x, y] == 1: #skosny prawy
        if (img_mag[x, y] > img_mag[x - 1, y + 1]) and (img_mag[x, y] > img_mag[x + 1, y - 1]):
          img_nonmax[x, y] = img_mag[x, y]
        else:
          img_nonmax[x, y] = 0
      elif matrix[x, y] == 2: #pionowy
        if (img_mag[x, y] > img_mag[x - 1, y]) and (img_mag[x, y] > img_mag[x + 1, y]):
          img_nonmax[x, y] = img_mag[x, y]
        else:
          img_nonmax[x, y] = 0
      elif matrix[x, y] == 3: #skosny lewy
        if (img_mag[x, y] > img_mag[x - 1, y - 1]) and (img_mag[x, y] > img_mag[x + 1, y + 1]):
          img_nonmax[x, y] = img_mag[x, y]
        else:
          img_nonmax[x, y] = 0
  return img_nonmax

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
# gauss
img_gauss = cv2.GaussianBlur(cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE), (3, 3), 0)
img_canny = cv2.Canny(img_gauss, 40, 180, None, 3, 1)
plt.imshow(img_canny, "gray")
plt.axis("off")
plt.show()

img_my = canny(cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE), 180, 40)
plt.imshow(img_my, "gray")
plt.axis("off")